In [5]:
!pip install spacy langdetect presidio-analyzer presidio-anonymizer rapidfuzz deep_translator unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.2 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 65.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 5.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=84c150634738c0f68c971115a859267d184417ab3b756e10640e284951faaad7
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [45]:
import json
import os
import re
import spacy
import unicodedata
from difflib import SequenceMatcher
from datetime import datetime
from langdetect import detect
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
from rapidfuzz import fuzz, process
from deep_translator import GoogleTranslator
from unidecode import unidecode

In [7]:
nlp_en = spacy.load("en_core_web_sm")

In [8]:
configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_sm"}],
}
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()
analyzer = AnalyzerEngine(nlp_engine=nlp_engine)

In [9]:
def detect_language(text):
    try:
        lang = detect(text)
        if lang == 'en':
            return 'English'
        elif lang == 'vi':
            return 'Vietnamese'
        else:
            return f'Other ({lang})'
    except:
        return 'Unknown'

In [15]:
def english_ner(text):
    doc = nlp_en(text)
    entities = []
    tracked_spans = set()  
    
    desired_spacy_labels = {'PERSON', 'ORG', 'GPE', 'LOC'}
    for ent in doc.ents:
        if ent.label_ in desired_spacy_labels:
            label = 'LOC' if ent.label_ in {'GPE', 'LOC'} else ent.label_
            span = (ent.start_char, ent.end_char)
            entities.append((ent.text, label, ent.start_char, ent.end_char))
            tracked_spans.add(span)
    
    presidio_entities = [
        "PHONE_NUMBER", "ADDRESS", "EMAIL_ADDRESS", "CREDIT_CARD", 
        "PERSON", "ORGANIZATION", "LOCATION", "URL"
    ]
    
    analyzer = AnalyzerEngine()
    
    presidio_results = analyzer.analyze(
        text=text, 
        language="en", 
        entities=presidio_entities,
        score_threshold=0.3  # Có thể fine tune lại 
    )
    
    label_mapping = {
        "PERSON": "PERSON",
        "ORGANIZATION": "ORG",
        "LOCATION": "LOC",
        "PHONE_NUMBER": "PHONE_NUMBER",
        "ADDRESS": "ADDRESS",
        "EMAIL_ADDRESS": "EMAIL_ADDRESS",
        "CREDIT_CARD": "CREDIT_CARD",
        "URL": "URL"
    }
    
    for result in presidio_results:
        entity_text = text[result.start:result.end]
        label = label_mapping.get(result.entity_type, result.entity_type)
        span = (result.start, result.end)
        
        overlap = False
        for tracked_start, tracked_end in tracked_spans:
            if (result.start <= tracked_end and result.end >= tracked_start):
                if label == "ADDRESS":
                    entities = [e for e in entities if not (e[2] <= result.end and e[3] >= result.start)]
                    tracked_spans.discard((tracked_start, tracked_end))
                    overlap = False
                    break
                else:
                    overlap = True
                    break
        
        if not overlap:
            entities.append((entity_text, label, result.start, result.end))
            tracked_spans.add(span)
    
    cc_patterns = [
        r'\b(?:\d{4}[-\s]?){3}\d{4}\b',          
        r'\b\d{4}[-\s]?\d{6}[-\s]?\d{5}\b',      
    ]
    
    for pattern in cc_patterns:
        for match in re.finditer(pattern, text):
            cc_text = match.group()
            start_idx = match.start()
            end_idx = match.end()
            
            overlap = False
            for tracked_start, tracked_end in tracked_spans:
                if (start_idx <= tracked_end and end_idx >= tracked_start):
                    overlap = True
                    break
            
            if not overlap:
                entities.append((cc_text, "CREDIT_CARD", start_idx, end_idx))
                tracked_spans.add((start_idx, end_idx))
    
    entities.sort(key=lambda x: x[2])
    return entities

In [112]:
def find_best_match(text, pattern):
    # Split text into phrases for matching
    phrases = re.findall(r'\w[\w ]*', text)
    best_match = process.extractOne(pattern, phrases, scorer=fuzz.ratio)
    if best_match and best_match[1] >= 50:  
        matched_text = best_match[0]
        start_idx = text.find(matched_text)
        end_idx = start_idx + len(matched_text)
        return matched_text, start_idx, end_idx
    return None

def map_entities_to_original(original_text, non_diacritics_text, english_entities):
    mapped_entities = []
    translator_en_to_vi = GoogleTranslator(source='en', target='vi')
    
    for entity_text, label, en_start, en_end in english_entities:
        if label == 'LOC':
            try:
                vi_entity_text = translator_en_to_vi.translate(entity_text)
                match = find_best_match(original_text, vi_entity_text)
                if match:
                    matched_text, start_idx, end_idx = match
                    mapped_entities.append((matched_text, label, start_idx, end_idx))
                else:
                    print(f"Location entity '{vi_entity_text}' not found with sufficient similarity")
            except Exception as e:
                print(f"Error translating '{entity_text}' to Vietnamese: {e}")
        else:
            # For non-LOC entities, search in non-diacritics text
            start_idx = non_diacritics_text.find(entity_text)
            if start_idx != -1:
                end_idx = start_idx + len(entity_text)
                original_entity_text = original_text[start_idx:end_idx]
                mapped_entities.append((original_entity_text, label, start_idx, end_idx))
            else:
                print(f"Entity '{entity_text}' not found in non_diacritics_text")
    
    return mapped_entities

def vietnamese_ner(text):
    original_text = text
    non_diacritics_text = remove_diacritics(text)
    
    translator = GoogleTranslator(source='vi', target='en')
    translated_text = translator.translate(text)  
    
    english_entities = english_ner(translated_text)
    mapped_entities = map_entities_to_original(original_text, non_diacritics_text, english_entities)
    
    return mapped_entities

In [113]:
def process_text(text):
    language = detect_language(text)
    print(f"Detected language: {language}")

    if language == 'English':
        entities = english_ner(text)
    elif language == 'Vietnamese':
        entities = vietnamese_ner(text)
    else:
        entities = []
        print(f"NER not supported for {language}")
        
    return entities

In [114]:
def entities_to_json(entities, output_dir="output", filename=None):
    os.makedirs(output_dir, exist_ok=True)
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"entities_{timestamp}.json"
    
    output_path = os.path.join(output_dir, filename)
    
    entities_json = [
        {
            "text": text,
            "label": label,
            "start": start,
            "end": end
        }
        for text, label, start, end in entities
    ]
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(entities_json, f, indent=4, ensure_ascii=False)
    
    return output_path

In [115]:
if __name__ == "__main__":
    text_en = """
    Nguyễn Thị Minh Anh, một kỹ sư phần mềm tại  FPT Software, hiện sống tại 123 đường Trần Hưng Đạo, Quận 1, Thành phố Hồ Chí Minh. Bạn có thể liên hệ với cô qua số điện thoại 0935123456 hoặc email minhanh.nguyen@fpt.com.vn. Số thẻ tín dụng của cô là 4532-7193-8214-5067, và cô thường sử dụng website https://minhanh.dev để giới thiệu các dự án của mình. Cô có cuộc họp với nhóm vào ngày 15 tháng 3 năm 2025, lúc 10 giờ sáng.

    Trong khi đó, Trần Văn Hùng, một chuyên gia tiếp thị, sống tại 456 đường Lê Lợi, Thành phố Huế, Thừa Thiên Huế. Số điện thoại của anh là 0918765432, và email là hung.tran@tu vanthitruong.vn. Anh sử dụng thẻ tín dụng số 6011-4729-3816-2458 cho các thanh toán. Website doanh nghiệp của anh là https://hungmarketing.vn, và anh sẽ tổ chức một hội thảo trực tuyến vào ngày 30 tháng 4 năm 2025. Văn phòng của anh nằm tại 789 Tòa nhà Kinh Doanh, Tầng 3, Thành phố Huế.
    
    Một liên hệ khác, Phạm Thị Hồng Nhung, là nhà thiết kế đồ họa, sống tại 321 đường Nguyễn Huệ, Thành phố Đà Nẵng. Thông tin liên hệ của cô bao gồm số điện thoại 0905123789 và email hongnhung.pham@thietkestudio.vn. Thẻ tín dụng của cô là 5467-3912-8046-1273, và danh mục đầu tư của cô có thể xem tại https://nhungdesigns.vn. Cô có lịch hẹn với khách hàng vào ngày mai lúc 2 giờ chiều.
    
    Cuối cùng, Lê Quốc Bảo, một cố vấn tài chính, sống tại 654 đường Hai Bà Trưng, Quận Hoàn Kiếm, Hà Nội. Số điện thoại của anh là 0987654321, và email là quocbao.le@taichinhgroup.vn. Số thẻ tín dụng của anh là 3782-914506-73254. Website chuyên nghiệp của anh là https://baofinance.vn, và anh có cuộc hẹn với khách hàng vào tuần tới, ngày 5 tháng 5 năm 2025, lúc 9 giờ sáng. Tất cả họ sẽ tham dự một hội nghị tại Khách sạn Grand, 1234 đường Hội Nghị, Quận 7, Thành phố Hồ Chí Minh, vào ngày 10 tháng 6 năm 2025.
    """
    
    entities_en = process_text(text_en)
    output_path = entities_to_json(entities_en)
    print(entities_en)
    print(f"Entities saved to: {output_path}")

Detected language: Vietnamese
[('Nguyen Thi Minh Anh', 'PERSON', 0, 19), ('FPT Software', 'ORG', 44, 56), ('Tran Hung', 'PERSON', 81, 90), ('District 1', 'LOC', 103, 113), ('Ho Chi Minh City', 'LOC', 115, 131), ('0935123456', 'PHONE_NUMBER', 170, 180), ('minhanh.nguyen@fpt.com.vn', 'EMAIL_ADDRESS', 190, 215), ('4532-7193-8214-5067', 'CREDIT_CARD', 243, 262), ('https://minhanh.dev', 'URL', 295, 314), ('Tran Van Hung', 'PERSON', 420, 433), ('Le Loi', 'LOC', 468, 474), ('Hue city', 'LOC', 483, 491), ('Thua Thien Hue', 'PERSON', 493, 507), ('0918765432', 'PHONE_NUMBER', 529, 539), ('hung.tr', 'URL', 554, 561), ('vanthitruong.vn', 'URL', 567, 582), ('6011-4729-3816-2458', 'CREDIT_CARD', 608, 627), ('https://hungmarketing.vn', 'URL', 666, 690), ('Hue city', 'LOC', 809, 817), ('Pham Thi Hong Nhung', 'PERSON', 850, 869), ('Da Nang City', 'LOC', 927, 939), ('hongnhung.pham@thietkestudio.vn', 'EMAIL_ADDRESS', 1008, 1039), ('5467-3912-8046-1273', 'CREDIT_CARD', 1060, 1079), ('https://nhungdesigns

In [34]:
Nguyễn Thị Minh Anh, một kỹ sư phần mềm tại Công ty FPT Software, hiện sống tại 123 đường Trần Hưng Đạo, Quận 1, TP. Hồ Chí Minh. Bạn có thể liên hệ với cô qua số điện thoại 0935123456 hoặc email minhanh.nguyen@fpt.com.vn. Số thẻ tín dụng của cô là 4532-7193-8214-5067, và cô thường sử dụng website https://minhanh.dev để giới thiệu các dự án của mình. Cô có cuộc họp với nhóm vào ngày 15 tháng 3 năm 2025, lúc 10 giờ sáng.

Trong khi đó, Trần Văn Hùng, một chuyên gia tiếp thị, sống tại 456 đường Lê Lợi, TP. Huế, Thừa Thiên Huế. Số điện thoại của anh là 0918765432, và email là hung.tran@tu vanthitruong.vn. Anh sử dụng thẻ tín dụng số 6011-4729-3816-2458 cho các thanh toán. Website doanh nghiệp của anh là https://hungmarketing.vn, và anh sẽ tổ chức một hội thảo trực tuyến vào ngày 30 tháng 4 năm 2025. Văn phòng của anh nằm tại 789 Tòa nhà Kinh Doanh, Tầng 3, TP. Huế.

Một liên hệ khác, Phạm Thị Hồng Nhung, là nhà thiết kế đồ họa, sống tại 321 đường Nguyễn Huệ, TP. Đà Nẵng. Thông tin liên hệ của cô bao gồm số điện thoại 0905123789 và email hongnhung.pham@thietkestudio.vn. Thẻ tín dụng của cô là 5467-3912-8046-1273, và danh mục đầu tư của cô có thể xem tại https://nhungdesigns.vn. Cô có lịch hẹn với khách hàng vào ngày mai lúc 2 giờ chiều.

Cuối cùng, Lê Quốc Bảo, một cố vấn tài chính, sống tại 654 đường Hai Bà Trưng, Quận Hoàn Kiếm, Hà Nội. Số điện thoại của anh là 0987654321, và email là quocbao.le@taichinhgroup.vn. Số thẻ tín dụng của anh là 3782-914506-73254. Website chuyên nghiệp của anh là https://baofinance.vn, và anh có cuộc hẹn với khách hàng vào tuần tới, ngày 5 tháng 5 năm 2025, lúc 9 giờ sáng. Tất cả họ sẽ tham dự một hội nghị tại Khách sạn Grand, 1234 đường Hội Nghị, Quận 7, TP. Hồ Chí Minh, vào ngày 10 tháng 6 năm 2025.

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (158178281.py, line 1)